In [74]:
import torch

In [75]:
def read_from_csv(path):
  f = open(path, 'r')
  lines = f.readlines()
  f.close()
  lines.pop(0)
  x1 = []
  x2 = []
  y = []

  for line in lines:
    _x1, _x2, _y = line.split(',')
    x1.append(float(_x1))
    x2.append(float(_x2))
    y.append(float(_y))
  return x1, x2, y


path = 'data_4.3.2.csv'
x1, x2, y = read_from_csv(path)

print(x1)
print(x2)
print(y)
print(len(y))


[9.076, 6.809, 8.256, 8.461, 6.783, 6.616, 5.845, 6.645, 5.791, 7.488, 7.296, 6.801, 4.988, 6.416, 6.502, 8.005, 6.541, 6.034, 5.077, 9.852, 5.237, 6.909, 7.251, 7.833, 6.725, 7.92, 7.86, 6.674, 6.201, 8.228, 8.196, 6.578, 8.306, 8.587, 9.765, 5.19, 9.406, 8.359, 7.71, 5.249, 6.84, 6.73, 9.763, 9.428, 7.419, 10.14, 6.707, 9.964, 6.931, 8.235, 8.338, 6.07, 8.521, 8.529, 8.232, 8.094, 9.176, 9.203, 5.119, 7.337, 7.521, 8.092, 4.847, 6.866, 7.515, 6.827, 9.022, 6.62, 8.388, 7.931, 7.052, 7.69, 7.624, 8.355, 9.973, 4.996, 9.196, 8.373, 5.498, 8.274, 8.278, 6.77, 8.331, 8.204, 4.935, 7.802, 5.638, 7.688, 5.293, 8.413, 4.991, 8.2, 7.356, 8.865, 6.912, 4.872, 6.075, 6.685, 7.197, 7.41, 7.054, 6.5, 6.076, 9.474, 7.053, 6.456, 6.539, 6.783, 9.375, 8.462, 9.842, 9.937, 8.143, 6.356, 8.127, 8.595, 7.943, 7.228, 7.07, 9.814, 6.678, 7.192, 7.221, 9.918, 5.899, 7.051, 8.391, 6.364, 6.253, 6.017, 6.563, 8.42, 7.463, 6.903, 5.826, 8.591, 9.402, 7.795, 9.303, 8.648, 7.097, 8.303, 8.188, 8.091, 7.433, 8

In [76]:
def h(tx, th):
  tz = torch.mm(tx, th)  # tensor(m, 1)
  tz = torch.sigmoid(tz)  # h(x) = g(z)
  return tz


def cost_func(tz, ty):
  cost = -ty*torch.log(tz)-(1-ty)*torch.log(1-tz)  # tensor(m, 1)
  cost = cost.sum()/m
  return cost


In [77]:
m = len(x1)

tx1 = torch.tensor(x1, dtype=torch.float32).reshape(m,1)
tx2 = torch.tensor(x2, dtype=torch.float32).reshape(m, 1)
ty = torch.tensor(y, dtype=torch.float32).reshape(m, 1)

tx3 = tx1**2
tx4 = tx2**2

tx0 = torch.ones(m,1)
tx = torch.concat([tx0, tx1, tx2, tx3, tx4], dim=1)

th = torch.zeros(5, 2, dtype=torch.float32)
th.requires_grad = True

alpha = 0.01
for iter in range(500):
  #hypothesis
  tz = h(tx, th)

  #cost
  cost = cost_func(tz, ty)

  print('ITER: %d => Cost: %f' % (iter, cost.item()))
  #print(th)
  cost.backward()

  with torch.no_grad():
    th += -alpha*th.grad
    th.grad.zero_()


ITER: 0 => Cost: 1.386294
ITER: 1 => Cost: 1.406319
ITER: 2 => Cost: 2.599927
ITER: 3 => Cost: 12.277456
ITER: 4 => Cost: 4.895922
ITER: 5 => Cost: 12.393349
ITER: 6 => Cost: 4.778862
ITER: 7 => Cost: 12.469970
ITER: 8 => Cost: 4.701300
ITER: 9 => Cost: 12.519300
ITER: 10 => Cost: 4.651102
ITER: 11 => Cost: 12.550890
ITER: 12 => Cost: 4.618653
ITER: 13 => Cost: 12.571460
ITER: 14 => Cost: 4.597226
ITER: 15 => Cost: 12.585367
ITER: 16 => Cost: 4.582448
ITER: 17 => Cost: 12.595341
ITER: 18 => Cost: 4.571592
ITER: 19 => Cost: 12.602998
ITER: 20 => Cost: 4.563051
ITER: 21 => Cost: 12.609304
ITER: 22 => Cost: 4.555863
ITER: 23 => Cost: 12.614803
ITER: 24 => Cost: 4.549484
ITER: 25 => Cost: 12.619815
ITER: 26 => Cost: 4.543588
ITER: 27 => Cost: 12.624538
ITER: 28 => Cost: 4.537989
ITER: 29 => Cost: 12.629072
ITER: 30 => Cost: 4.532575
ITER: 31 => Cost: 12.633479
ITER: 32 => Cost: 4.527287
ITER: 33 => Cost: 12.637796
ITER: 34 => Cost: 4.522095
ITER: 35 => Cost: 12.642034
ITER: 36 => Cost: 4.5

In [78]:
with torch.no_grad():
  tz = h(tx, th)
  tz[tz >= 0.5] = 1.0
  tz[tz < 0.5] = 0.0
  accuracy = (tz == ty).sum()/m
  print('ACCURACY: %.2f' % (accuracy.item()*100))


ACCURACY: 100.00
